# Pipeline Setup

In [1]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
data = load_boston()
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'])

In [2]:
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge

In [3]:
scaler = StandardScaler()
pca = PCA()
ridge = Ridge()

In [4]:
X_train = scaler.fit_transform(X_train)
X_train = pca.fit_transform(X_train)
ridge.fit(X_train, y_train)

Ridge()

In [5]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('reduce_dim', PCA()),
        ('regressor', Ridge())
        ])

In [6]:
pipe = pipe.fit(X_train, y_train)
print('Testing score: ', pipe.score(X_test, y_test))

Testing score:  -3765.156902454027


It is also possible to index the pipeline to access a specific element and retrieve a single value, for example the explained variance in the PCA step:

In [7]:
print(pipe.steps[1][1].explained_variance_)

[1.0026455 1.0026455 1.0026455 1.0026455 1.0026455 1.0026455 1.0026455
 1.0026455 1.0026455 1.0026455 1.0026455 1.0026455 1.0026455]


In [8]:
# The above numbers should be different...

# Pipeline Tuning (Base Version)

In [9]:
import numpy as np
n_features_to_test = np.arange(1, 11)

In [12]:
alpha_to_test = 2.0**np.arange(-6, +6)
alpha_to_test

array([1.5625e-02, 3.1250e-02, 6.2500e-02, 1.2500e-01, 2.5000e-01,
       5.0000e-01, 1.0000e+00, 2.0000e+00, 4.0000e+00, 8.0000e+00,
       1.6000e+01, 3.2000e+01])

In [13]:
params = {'reduce_dim__n_components': n_features_to_test,\
              'regressor__alpha': alpha_to_test}

It is worth remarking the convention adopted to name the parameters: name of the pipeline step, followed by a double underscore (__), then finally the name of the parameter within the step. The optimization is invoked as follows:

In [14]:
from sklearn.model_selection import GridSearchCV
gridsearch = GridSearchCV(pipe, params, verbose=1).fit(X_train, y_train)
print('Final score is: ', gridsearch.score(X_test, y_test))

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Final score is:  -11539.143751400004


In [15]:
gridsearch.best_params_

{'reduce_dim__n_components': 10, 'regressor__alpha': 8.0}

# Pipeline Tuning (Advanced Version)

In [16]:
scalers_to_test = [StandardScaler(), RobustScaler(), QuantileTransformer()]

In [17]:
params = {'scaler': scalers_to_test,
        'reduce_dim__n_components': n_features_to_test,\
        'regressor__alpha': alpha_to_test}

In [18]:
params = [
        {'scaler': scalers_to_test,
         'reduce_dim': [PCA()],
         'reduce_dim__n_components': n_features_to_test,\
         'regressor__alpha': alpha_to_test},

        {'scaler': scalers_to_test,
         'reduce_dim': [SelectKBest(f_regression)],
         'reduce_dim__k': n_features_to_test,\
         'regressor__alpha': alpha_to_test}
        ]

In [21]:
gridsearch = GridSearchCV(pipe, params, verbose=1).fit(X_train, y_train)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


F:\Anaconda\envs\lhl\lib\site-packages\sklearn\preprocessing\_data.py:2615: UserWarning: n_quantiles (1000) is greater than the total number of samples (303). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
F:\Anaconda\envs\lhl\lib\site-packages\sklearn\preprocessing\_data.py:2615: UserWarning: n_quantiles (1000) is greater than the total number of samples (303). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
F:\Anaconda\envs\lhl\lib\site-packages\sklearn\preprocessing\_data.py:2615: UserWarning: n_quantiles (1000) is greater than the total number of samples (303). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
F:\Anaconda\envs\lhl\lib\site-packages\sklearn\preprocessing\_data.py:2615: UserWarning: n_quantiles (1000) is greater than the total number of samples (303). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
F:\Anaconda\envs\lhl\lib\site-packages\sklearn\preprocessing\_data.py:2615: UserWarning: n_q

In [22]:
print('Final score is: ', gridsearch.score(X_test, y_test))

Final score is:  -3800.0843431946205


In [23]:
gridsearch.best_params_

{'reduce_dim': SelectKBest(score_func=<function f_regression at 0x0000021785DB49D8>),
 'reduce_dim__k': 10,
 'regressor__alpha': 4.0,
 'scaler': StandardScaler()}